<a href="https://colab.research.google.com/github/surajgajul/Cocktails_model/blob/main/bevVGG16(Test).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
train_datagen_aug = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 20,
                                   validation_split=0.2,
                                   horizontal_flip = True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

training_set_aug = train_datagen_aug.flow_from_directory(directory= '/content/drive/MyDrive/Bev_Dataset',
                                               target_size=(224, 224), # As we choose 64*64 for our convolution model
                                               batch_size=50,
                                               class_mode='categorical',
                                               subset="training"
                                               )
valid_set = train_datagen_aug.flow_from_directory(directory= '/content/drive/MyDrive/Bev_Dataset',
                                            target_size=(224, 224),
                                            batch_size=50,
                                            class_mode='categorical',
                                            subset="validation")

Mounted at /content/drive
Found 669 images belonging to 7 classes.
Found 164 images belonging to 7 classes.


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0025, patience=2, verbose=1)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense ,Dropout,Activation,Flatten,Conv2D,MaxPooling2D

In [ ]:
from keras.layers import Input

In [ ]:
base_model = VGG16(include_top=False,
                         input_shape=(224,224,3),
                         weights='imagenet',
                         pooling="avg"
                     )

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
from tensorflow.keras.regularizers import l2

In [ ]:
base_model.trainable = False
image_input = Input(shape=(224, 224, 3))

x = base_model(image_input,training = False)

x = Dense(256,activation = "relu")(x)
x = Dropout(0.2)(x)

x = Dense(128,activation = "relu")(x)
x = Dropout(0.2)(x)

image_output = Dense(7,activation="softmax")(x)

model = Model(image_input,image_output)
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
hist = model.fit(training_set_aug,validation_data = valid_set, epochs=10, callbacks=[callback])
model.save('/content/drive/MyDrive/Bev_Dataset/model.h5')

Epoch 1/10
14/14 [==============================] - 492s 37s/step - loss: 1.9860 - accuracy: 0.1883 - val_loss: 1.7858 - val_accuracy: 0.4512
Epoch 2/10
14/14 [==============================] - 12s 871ms/step - loss: 1.7915 - accuracy: 0.2945 - val_loss: 1.6397 - val_accuracy: 0.4695
Epoch 3/10
14/14 [==============================] - 12s 882ms/step - loss: 1.6301 - accuracy: 0.4260 - val_loss: 1.4332 - val_accuracy: 0.5000
Epoch 4/10
14/14 [==============================] - 13s 900ms/step - loss: 1.4569 - accuracy: 0.4858 - val_loss: 1.2435 - val_accuracy: 0.6159
Epoch 5/10
14/14 [==============================] - 12s 925ms/step - loss: 1.3647 - accuracy: 0.5007 - val_loss: 1.1400 - val_accuracy: 0.6341
Epoch 6/10
14/14 [==============================] - 12s 919ms/step - loss: 1.2511 - accuracy: 0.5650 - val_loss: 1.0512 - val_accuracy: 0.6951
Epoch 7/10
14/14 [==============================] - 14s 1s/step - loss: 1.1578 - accuracy: 0.5755 - val_loss: 1.0646 - val_accuracy: 0.6707
Epo

In [ ]:
from keras.models import load_model
import os
from keras.preprocessing import image
import numpy as np
model=load_model('/content/drive/MyDrive/Bev_Dataset/model.h5')
test_folder=os.listdir("/content/drive/MyDrive/Bev_Test")

In [ ]:
y_true=[]
for i in test_folder:
  for j in range(len(os.listdir(f"/content/drive/MyDrive/Bev_Test/{i}"))):
    y_true.append(i)

In [ ]:
import tensorflow
y_predicted=[]
for i in test_folder:
  for j in os.listdir(f"/content/drive/MyDrive/Bev_Test/{i}"):
    img = tensorflow.keras.utils.load_img(f"/content/drive/MyDrive/Bev_Test/{i}/{j}", target_size=(224,224))
    x = tensorflow.keras.utils.img_to_array(img)
    x=x/255
    x=x.reshape(224,224,3)
    x = np.expand_dims(x, axis=0)
    # with model_graph.as_default():
        #     with tf_session.as_default():
    predi=model.predict(x)
    print(predi)
    classes_x=np.argmax(predi)
        
    print(classes_x)
    classes=["beer","cocktails","coffee","soft_drinks","sports_drink","water","wine"]    
    MaxPosition=np.argmax(predi)  
    prediction_label=prediction_label=classes[MaxPosition]
    y_predicted.append(prediction_label)

1/1 [==============================] - 3s 3s/step
[[0.2409411  0.31338134 0.03410442 0.07203254 0.03380343 0.05114043
  0.25459668]]
1
1/1 [==============================] - 0s 42ms/step
[[0.2843577  0.09259771 0.08262192 0.08418445 0.10478298 0.2646573
  0.08679795]]
0
1/1 [==============================] - 0s 56ms/step
[[0.01687933 0.8416888  0.00493523 0.01268966 0.0023234  0.00247104
  0.1190125 ]]
1
1/1 [==============================] - 0s 35ms/step
[[0.06142414 0.5289402  0.02492319 0.00728888 0.00536643 0.00906174
  0.36299536]]
1
1/1 [==============================] - 0s 22ms/step
[[0.00534267 0.00846934 0.94035035 0.0024591  0.00523421 0.00580171
  0.03234261]]
2
1/1 [==============================] - 0s 24ms/step
[[0.00408861 0.02865683 0.93184006 0.00180064 0.00167051 0.00122055
  0.03072279]]
2
1/1 [==============================] - 0s 23ms/step
[[0.10345983 0.00317599 0.00384717 0.09739624 0.61000985 0.17529508
  0.00681576]]
4
1/1 [==============================] - 0s 26

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
print(y_predicted)
print(y_true)
cm = confusion_matrix(y_true, y_predicted)
cm

['cocktails', 'beer', 'cocktails', 'cocktails', 'coffee', 'coffee', 'sports_drink', 'sports_drink', 'soft_drinks', 'coffee', 'water', 'water', 'wine', 'cocktails']
['beer', 'beer', 'cocktails', 'cocktails', 'coffee', 'coffee', 'sports_drink', 'sports_drink', 'soft_drinks', 'soft_drinks', 'water', 'water', 'wine', 'wine']


array([[1, 1, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1]])